In [ ]:
from keras.utils import to_categorical
from tensorflow.keras.utils import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.callbacks import EarlyStopping
import os
import pandas as pd
import numpy as np

In [ ]:
pip install ipywidgets

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------- ----------------- 1.3/2.3 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [ ]:
TRAIN_DIR = r'"C:/Users/khang/OneDrive/Desktop/Machine_Learning/images/train"'
TEST_DIR = r'"C:/Users/khang/OneDrive/Desktop/Machine_Learning/images/test"'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [ ]:
print(train)

                                             image     label
0            D:\Downloads\images\train\angry\0.jpg     angry
1            D:\Downloads\images\train\angry\1.jpg     angry
2           D:\Downloads\images\train\angry\10.jpg     angry
3        D:\Downloads\images\train\angry\10002.jpg     angry
4        D:\Downloads\images\train\angry\10016.jpg     angry
...                                            ...       ...
28816  D:\Downloads\images\train\surprise\9969.jpg  surprise
28817  D:\Downloads\images\train\surprise\9985.jpg  surprise
28818  D:\Downloads\images\train\surprise\9990.jpg  surprise
28819  D:\Downloads\images\train\surprise\9992.jpg  surprise
28820  D:\Downloads\images\train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [ ]:
print(test)
print(test['image'])

                                                 image     label
0       D:\Downloads\images\validation\angry\10052.jpg     angry
1       D:\Downloads\images\validation\angry\10065.jpg     angry
2       D:\Downloads\images\validation\angry\10079.jpg     angry
3       D:\Downloads\images\validation\angry\10095.jpg     angry
4       D:\Downloads\images\validation\angry\10121.jpg     angry
...                                                ...       ...
7061  D:\Downloads\images\validation\surprise\9806.jpg  surprise
7062  D:\Downloads\images\validation\surprise\9830.jpg  surprise
7063  D:\Downloads\images\validation\surprise\9853.jpg  surprise
7064  D:\Downloads\images\validation\surprise\9878.jpg  surprise
7065   D:\Downloads\images\validation\surprise\993.jpg  surprise

[7066 rows x 2 columns]
0         D:\Downloads\images\validation\angry\10052.jpg
1         D:\Downloads\images\validation\angry\10065.jpg
2         D:\Downloads\images\validation\angry\10079.jpg
3         D:\Downloads\i

In [ ]:
from tqdm import tqdm


In [ ]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [ ]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

100%|██████████| 28821/28821 [07:19<00:00, 65.54it/s]


In [ ]:
test_features = extract_features(test['image'])

100%|██████████| 7066/7066 [01:51<00:00, 63.31it/s]


In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [ ]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
model = Sequential()

# Block 1: CONV (64 filters, 3x3 kernel, ReLU activation)
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(48,48,1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 2: CONV (128 filters, 3x3 kernel, ReLU activation)
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 3: CONV (256 filters, 3x3 kernel, ReLU activation)
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 4: CONV (512 filters, 3x3 kernel, ReLU activation)
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Block 5: CONV (512 filters, 3x3 kernel, ReLU activation)
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# Fully Connected Layers
model.add(Flatten())  # Làm phẳng các feature maps
model.add(Dense(4096, activation='relu'))  # FC1
model.add(Dropout(0.5))  # Dropout để tránh overfitting
model.add(Dense(4096, activation='relu'))  # FC2
model.add(Dropout(0.5))  # Dropout để tránh overfitting

# Output Layer (Sử dụng softmax cho phân loại nhiều lớp)
model.add(Dense(units=7, activation='softmax'))


c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',       # Theo dõi chỉ số 'val_loss' (loss trên tập validation)
    patience=5,               # Số epoch không cải thiện trước khi dừng
    restore_best_weights=True,  # Phục hồi trọng số tốt nhất của mô hình
    min_delta=0.001           # Mức cải thiện tối thiểu (ở đây là 0.001)
)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]  # Thêm Early Stopping vào đây
)

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 1071s 5s/step - accuracy: 0.2338 - loss: 3.4766 - val_accuracy: 0.2583 - val_loss: 1.8111
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 931s 4s/step - accuracy: 0.2482 - loss: 1.8147 - val_accuracy: 0.2583 - val_loss: 1.8121
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 793s 4s/step - accuracy: 0.2478 - loss: 1.8114 - val_accuracy: 0.2583 - val_loss: 1.8092
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 1309s 6s/step - accuracy: 0.2490 - loss: 1.8099 - val_accuracy: 0.2583 - val_loss: 1.8089
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 1377s 6s/step - accuracy: 0.2496 - loss: 1.8129 - val_accuracy: 0.2583 - val_loss: 1.8094
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 536s 2s/step - accuracy: 0.2483 - loss: 1.8102 - val_accuracy: 0.2583 - val_loss: 1.8092
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 431s 2s/step - accuracy: 0.2495 - loss: 1.8113 - val_accuracy: 0.2583 - val_loss: 1.8104
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 5759s 26s/step - accuracy: 0.2503 - loss: 1.81

In [ ]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from tensorflow.keras.utils import plot_model
print(model.summary())

# Visualize the model architecture
plot_model(
    model,
    to_file='model_architecture.png',  # Save the model diagram as an image
    show_shapes=True,                  # Display input/output shapes
    show_layer_names=True              # Display layer names
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │     2,101,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │        28,679 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 100,874,327 (384.81 MB)

 Trainable params: 33,624,775 (128.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 67,249,552 (256.54 MB)

None
You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
file_path = 'C:/Users/khang/OneDrive/Desktop/Machine_Learning'
# Lưu model bằng file json
model_json = model.to_json()
with open(os.path.join(file_path, "emotiondetector_vcg16.json"), 'w') as json_file:
    json_file.write(model_json)

# Lưu trọng số của model
model.save(os.path.join(file_path, "emotiondetector_vcg16.h5"))